In [0]:
from google.colab import drive
drive.mount("/content/drive")
import pandas as pd
import glob
import os
import numpy as np
from sklearn.ensemble import ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import scale
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import log_loss, accuracy_score, roc_auc_score, roc_curve, auc
from sklearn.model_selection import train_test_split, GridSearchCV
import pandas as pd
from sklearn.utils import resample

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
!pip install lime

In [0]:
import lime
import lime.lime_tabular

In [0]:
#!unzip -uq "/content/drive/My Drive/Curious Zucchini/2016_GCP_POA_for_AUS_short-header.zip" -d "/content/drive/My Drive/Curious Zucchini/" 

In [0]:
df_main = pd.read_excel("/content/drive/My Drive/Curious Zucchini/main-dataset2.xlsx")

In [0]:
df_main["Calendar Year"].value_counts()

2018    8075
2017    8021
2016    7973
2015    7967
2014    7923
2013    7892
2012    7850
2011    7811
2010    7746
2009    7721
2008    7710
Name: Calendar Year, dtype: int64

In [0]:
df_main2 = df_main[df_main["Calendar Year"] == 2016]

In [0]:
#download data from https://datapacks.censusdata.abs.gov.au/datapacks/ for ABS Census 2016 data (ALL of Australia) then just get the folder "2016 Census GCP Postal Areas for AUST" save it in the same folder
#change path where the file "2016 Census GCP Postal Areas for AUST" will be saved
path =  "/content/drive/My Drive/Curious Zucchini/2016 Census GCP Postal Areas for AUST"

all_files = glob.glob(path + "/*.csv")

In [0]:
dictionary_df = {os.path.splitext(os.path.basename(f))[0] : pd.read_csv(f) for f in all_files}

In [0]:
df_main2["Postcode_POA"] = "POA" + df_main2["Postcode"].astype("str")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [0]:
df_main3 = df_main2.reset_index().drop(["index"], axis=1)
ACARA_ID_index = df_main3[['ACARA SML ID']]

In [0]:
for k, v in dictionary_df.items():
    df_main4 = df_main3.merge(dictionary_df[k], left_on="Postcode_POA", right_on="POA_CODE_2016", how="left")

In [0]:
print("starting dataframe " + str(df_main2.shape) + " joined dataframe " + str(df_main4.shape))

starting dataframe (7973, 33) joined dataframe (7973, 234)


In [0]:
df_main4.select_dtypes(include=["O"]).columns

Index(['School Name', 'Suburb', 'State', 'School Sector', 'School Type',
       'Campus Type', 'Rolled Reporting Description', 'School URL',
       'Governing Body', 'Governing Body URL', 'Year Range', 'Geolocation',
       'Postcode_POA', 'POA_CODE_2016'],
      dtype='object')

In [0]:
df_main4.select_dtypes(include=["O"]).head(1)

,School Name,Suburb,State,School Sector,School Type,Campus Type,Rolled Reporting Description,School URL,Governing Body,Governing Body URL,Year Range,Geolocation,Postcode_POA,POA_CODE_2016
0,Corpus Christi Catholic School,Bellerive,TAS,Catholic,Primary,School Single Entity,Individual Reporting,http://www.corpuschristi.tas.edu.au,Tasmanian Catholic Education Office,http://www.catholic.tas.edu.au,Prep-6,Inner Regional,POA7018,POA7018


In [0]:
list_to_drop = ["Postcode_POA",
                "POA_CODE_2016",
                "School Name",
                "Rolled Reporting Description",
                "School URL",
                "Governing Body URL",
                "Calendar Year",
                "ACARA SML ID",
                "AGE ID"]

In [0]:
df_main5 = df_main4.drop(list_to_drop, axis=1)

In [0]:
df_main6 = pd.get_dummies(df_main5)

In [0]:
df_main6.head(1)

,Hubs - Target,Postcode,ICSEA,Bottom SEA Quarter (%),Lower Middle SEA Quarter (%),Upper Middle SEA Quarter (%),Top SEA Quarter (%),Teaching Staff,Full Time Equivalent Teaching Staff,Non-Teaching Staff,Full Time Equivalent Non-Teaching Staff,Total Enrolments,Girls Enrolments,Boys Enrolments,Full Time Equivalent Enrolments,Indigenous Enrolments (%),Language Background Other Than English (%),M_Mng_0,M_Mng_1_15,M_Mng_16_24,M_Mng_25_34,M_Mng_35_39,M_Mng_40,M_Mng_41_48,M_Mng_49over,M_Mng_hours_NS,M_Mng_Tot,M_Pro_0,M_Pro_1_15,M_Pro_16_24,M_Pro_25_34,M_Pro_35_39,M_Pro_40,M_Pro_41_48,M_Pro_49over,M_Pro_hours_NS,M_Pro_Tot,M_TTW_0,M_TTW_1_15,M_TTW_16_24,...,Year Range_PP-9,Year Range_PP-10,Year Range_PP-11,Year Range_2020-08-01 00:00:00,Year Range_2020-12-01 00:00:00,Year Range_2020-09-01 00:00:00,Year Range_PP-8,Year Range_PP-7,"Year Range_U, 3-7",Year Range_R-6,"Year Range_U, R-12","Year Range_U, 6-12","Year Range_U, R-6","Year Range_U, 7-12",Year Range_R-9,Year Range_R-5,Year Range_R-10,Year Range_R-11,Year Range_P-9,Year Range_P-10,Year Range_T-11,Year Range_2020-11-03 00:00:00,"Year Range_U, Prep-9",Year Range_PP-4,Year Range_2020-11-09 00:00:00,Year Range_PP-5,Year Range_PP-1,Year Range_Prep-7,Year Range_2020-11-10 00:00:00,Year Range_2020-11-01 00:00:00,Year Range_PP-2,Year Range_K-1,Year Range_7,Year Range_Prep-1,Year Range_2020-09-07 00:00:00,Geolocation_Inner Regional,Geolocation_Major Cities,Geolocation_Outer Regional,Geolocation_Remote,Geolocation_Very Remote
0,0,7018,1061.0,11.0,26.0,32.0,31.0,29.0,20.8,16.0,9.7,398.0,194.0,204.0,398.0,2.0,1.0,18.0,22.0,20.0,27.0,100.0,174.0,111.0,240.0,6.0,708.0,36.0,46.0,40.0,73.0,238.0,205.0,106.0,145.0,8.0,894.0,20.0,42.0,28.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [0]:
imputer = SimpleImputer(strategy = "median")

In [0]:
imputer.fit(df_main6)

SimpleImputer(add_indicator=False, copy=True, fill_value=None,
              missing_values=nan, strategy='median', verbose=0)

In [0]:
df_main7 = pd.DataFrame(imputer.transform(df_main6), columns=df_main6.columns)

In [0]:
True_table = df_main7[df_main7["Hubs - Target"] == 1]
False_table = df_main7[df_main7["Hubs - Target"] == 0]

In [0]:
Not_Hub_Downsample = resample(False_table, replace = False, n_samples = len(True_table), random_state = 42) 

In [0]:
downsampled_df = pd.concat([True_table, Not_Hub_Downsample])

In [0]:
def variance_threshold_selector(data, threshold=0.8):
    selector = VarianceThreshold(threshold)
    selector.fit(data)
    return data[data.columns[selector.get_support(indices=True)]]

In [0]:
target = downsampled_df["Hubs - Target"]
base_vars = downsampled_df.drop(["Hubs - Target"], axis=1)

In [0]:
target.value_counts()

0.0    73
1.0    73
Name: Hubs - Target, dtype: int64

In [0]:
base_vars2 = variance_threshold_selector(data= base_vars, threshold=0.99)

In [0]:
base_vars2.shape

(146, 216)

In [0]:
correlation_matrix = base_vars2.corr().abs()
count_list = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(np.bool))
highly_corr_vars = [column for column in count_list.columns if any(count_list[column] > 0.99)]

In [0]:
base_vars3 = base_vars2.drop(base_vars2[highly_corr_vars], axis=1)

In [0]:
base_vars3_5 = scale(base_vars3)

In [0]:
pca = PCA(n_components=4)
pca.fit(base_vars3_5)
pca.explained_variance_ratio_.cumsum()
base_vars4 = pca.fit_transform(base_vars3_5)

In [0]:
x_train, x_test, y_train, y_test = train_test_split(base_vars4, target, test_size=0.1, random_state=1)

In [0]:
lr_grid = {
    "C": np.logspace(-3,3,7), 
    "penalty":["l2"]
    } 
lr_fit=LogisticRegression()
lr_fit_crossVal=GridSearchCV(lr_fit, lr_grid, cv = 10)
lr_fit_crossVal.fit(x_train, y_train)

GridSearchCV(cv=10, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03]),
                         'penalty': ['l2']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [0]:
lr_fit_crossVal.best_params_

{'C': 0.01, 'penalty': 'l2'}

In [0]:
lr_fit_crossVal.best_score_

0.7104395604395605

In [0]:
probabilities = lr_fit_crossVal.predict_proba(x_test)

In [0]:
roc_auc_score(y_test, probabilities[:,1])

0.7142857142857143

In [0]:
fpr, tpr, thresholds = roc_curve(y_test, probabilities[:,1])

In [0]:
auc(fpr, tpr)

0.7142857142857143

In [0]:
lr_fit_crossVal.score

<bound method BaseSearchCV.score of GridSearchCV(cv=10, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03]),
                         'penalty': ['l2']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)>

In [0]:
columns_to_keep = list(base_vars3.columns)

In [0]:
feat_df = pd.DataFrame()
feat_df['cols'] = pd.Series(columns_to_keep)
feat_df.to_csv("/content/drive/My Drive/Curious Zucchini/features.csv")

## Score entire dataset

In [0]:
scoring_dataset = pca.fit_transform(df_main7[columns_to_keep])

In [0]:
scored_values = lr_fit_crossVal.predict_proba(scoring_dataset)[:,1]

In [0]:
df_main8 = df_main7[columns_to_keep]

In [0]:
df_main8["probability"] = scored_values

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [0]:
df_main8_ACARAID = df_main8.join(ACARA_ID_index)
#work hrs 1 to 49 up - Males
df_main8_ACARAID["SUM OF MALE WORKING HRS 1 to 49"] = df_main8_ACARAID["M_Tot_1_15"]+df_main8_ACARAID["M_Tot_16_24"]+df_main8_ACARAID["M_Tot_35_39"]+df_main8_ACARAID["M_Tot_40"]+df_main8_ACARAID["M_Tot_41_48"]+df_main8_ACARAID["M_Tot_49over"]
#all recorded work hrs, 1 to 49, 0 and NS - Males
df_main8_ACARAID["SUM OF MALE WORKING HRS 0 to NS"]= (df_main8_ACARAID["M_Tot_1_15"]+df_main8_ACARAID["M_Tot_16_24"]+df_main8_ACARAID["M_Tot_35_39"]+df_main8_ACARAID["M_Tot_40"]+df_main8_ACARAID["M_Tot_41_48"]+df_main8_ACARAID["M_Tot_49over"]+df_main8_ACARAID["M_Tot_0"]+df_main8_ACARAID["M_Tot_hours_NS"])
#percent of work hrs for males
df_main8_ACARAID["PERCENT OF MALE WORKING HOURS"]= df_main8_ACARAID["SUM OF MALE WORKING HRS 1 to 49"]/df_main8_ACARAID["SUM OF MALE WORKING HRS 0 to NS"]
#work hrs 1 to 49 up - Females
df_main8_ACARAID["SUM OF FEMALE HRS 1 to 49"] = df_main8_ACARAID["F_Tot_1_15"]+df_main8_ACARAID["F_Tot_16_24"]+df_main8_ACARAID["F_Tot_35_39"]+df_main8_ACARAID["F_Tot_40"]+df_main8_ACARAID["F_Tot_41_48"]+df_main8_ACARAID["F_Tot_49over"]
#all recorded work hrs, 1 to 49, 0 and NS - Females
df_main8_ACARAID["SUM OF FEMALE HRS 0 to NS"] = (df_main8_ACARAID["F_Tot_1_15"]+df_main8_ACARAID["F_Tot_16_24"]+df_main8_ACARAID["F_Tot_35_39"]+df_main8_ACARAID["F_Tot_40"]+df_main8_ACARAID["F_Tot_41_48"]+df_main8_ACARAID["F_Tot_49over"]+df_main8_ACARAID["F_Tot_0"]+df_main8_ACARAID["F_Tot_hours_NS"])
#percent of work hrs for Females
df_main8_ACARAID["PERCENT OF FEMALE WORKING HOURS"] = df_main8_ACARAID["SUM OF FEMALE HRS 1 to 49"]/df_main8_ACARAID["SUM OF FEMALE HRS 0 to NS"]

In [0]:
df_main8_ACARAID.head(1)

,Postcode,ICSEA,Bottom SEA Quarter (%),Lower Middle SEA Quarter (%),Upper Middle SEA Quarter (%),Top SEA Quarter (%),Teaching Staff,Full Time Equivalent Teaching Staff,Non-Teaching Staff,Full Time Equivalent Non-Teaching Staff,Total Enrolments,Girls Enrolments,Boys Enrolments,Indigenous Enrolments (%),Language Background Other Than English (%),M_Mng_0,M_Mng_1_15,M_Mng_16_24,M_Mng_25_34,M_Mng_35_39,M_Mng_40,M_Mng_41_48,M_Mng_49over,M_Mng_hours_NS,M_Mng_Tot,M_Pro_0,M_Pro_1_15,M_Pro_16_24,M_Pro_25_34,M_Pro_35_39,M_Pro_40,M_Pro_41_48,M_Pro_49over,M_Pro_hours_NS,M_Pro_Tot,M_TTW_0,M_TTW_1_15,M_TTW_16_24,M_TTW_25_34,M_TTW_35_39,...,F_MOD_hours_NS,F_MOD_Tot,F_Lab_0,F_Lab_1_15,F_Lab_16_24,F_Lab_25_34,F_Lab_35_39,F_Lab_40,F_Lab_41_48,F_Lab_49over,F_Lab_hours_NS,F_Lab_Tot,F_ID_NS_0,F_ID_NS_1_15,F_ID_NS_16_24,F_ID_NS_25_34,F_ID_NS_35_39,F_ID_NS_40,F_ID_NS_41_48,F_ID_NS_49over,F_ID_NS_hours_NS,F_ID_NS_Tot,F_Tot_0,F_Tot_1_15,F_Tot_16_24,F_Tot_25_34,F_Tot_35_39,F_Tot_40,F_Tot_41_48,F_Tot_49over,F_Tot_hours_NS,F_Tot_Tot,probability,ACARA SML ID,SUM OF MALE WORKING HRS 1 to 49,SUM OF MALE WORKING HRS 0 to NS,PERCENT OF MALE WORKING HOURS,SUM OF FEMALE HRS 1 to 49,SUM OF FEMALE HRS 0 to NS,PERCENT OF FEMALE WORKING HOURS
0,7018.0,1061.0,11.0,26.0,32.0,31.0,29.0,20.8,16.0,9.7,398.0,194.0,204.0,2.0,1.0,18.0,22.0,20.0,27.0,100.0,174.0,111.0,240.0,6.0,708.0,36.0,46.0,40.0,73.0,238.0,205.0,106.0,145.0,8.0,894.0,20.0,42.0,28.0,84.0,218.0,...,0.0,33.0,16.0,96.0,55.0,47.0,24.0,23.0,4.0,9.0,12.0,270.0,7.0,6.0,9.0,9.0,16.0,11.0,4.0,0.0,15.0,76.0,243.0,759.0,731.0,871.0,1006.0,535.0,278.0,309.0,58.0,4796.0,2.423528e-291,40000,3984.0,4191.0,0.950608,3618.0,3919.0,0.923195


In [0]:
df_main8_ACARAID.to_csv("/content/drive/My Drive/Curious Zucchini/ACARAID_PERCENT_WH.csv")